- La idea de este proyecto es subir un csv para limpiar.

In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [32]:
# cargamos el dataframe creado:
df_excel = pd.read_excel("C:/Users/isaw9/OneDrive/Escritorio/ADALAB/Modulo_3/files/New_clothesforyou.xlsx", index_col = 0)
df_excel.head()

,id,producto,precio (€),descuento (%),color,pais_origen,temporada2025,ventas_unidad,inventario,marca,estacion
NaN,1,Camiseta Fresh Breeze,23.75,10,azul,ES,Yes,34,213,A,2
NaN,2,Polo Urban Fit,45.6,15,rojo,IT,No,15,45,B,2
NaN,3,Blusa Soft Elegance,78.5,20,NaN,FR,No,12,3,C,3
NaN,4,Camisa Denim Classic,12.3,5,verde,PR,Si,78,534,D,2
NaN,5,Crop Top Sunset Glow,41.4,5,nergo,AL,No,65,NaN,A,3


In [ ]:
# Estructura de limpieza de datos:

# - Estudio de mis datos: Saber que datos son, a que corresponden, columnas...
# - Finalidad y preguntas del estudio.
# - Limpieza de datos adecuada a la finalidad, con criterio y justificaciones de cambios.

In [33]:
df_excel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115 entries, nan to nan
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             114 non-null    object
 1   producto       115 non-null    object
 2   precio (€)     110 non-null    object
 3   descuento (%)  110 non-null    object
 4   color          103 non-null    object
 5   pais_origen    107 non-null    object
 6   temporada2025  104 non-null    object
 7   ventas_unidad  109 non-null    object
 8   inventario     99 non-null     object
 9   marca          102 non-null    object
 10  estacion       95 non-null     object
dtypes: object(11)
memory usage: 10.8+ KB


In [34]:
# Tengo un dataframe de una tienda de ropa, que recoge diferentes datos de cada prenda. 
# Tenemos estas columnas:

df_excel.columns

Index(['id', 'producto', 'precio (€)', 'descuento (%)', 'color', 'pais_origen',
       'temporada2025', 'ventas_unidad', 'inventario', 'marca', 'estacion'],
      dtype='object')

In [ ]:
# todas de tipo objeto, por lo que las columnas numéricas no categóricas, convendrá convertirlas a int/float llegado el momento.

In [35]:
# Haciendo una valoraicón de calores únicos:

df_excel.nunique()

id               113
producto         100
precio (€)        79
descuento (%)     12
color             10
pais_origen        8
temporada2025      7
ventas_unidad     29
inventario        48
marca             13
estacion           9
dtype: int64

In [ ]:
# OBJETIVOS DEL ESTUDIO

# 1.-Conocer el valor de ventas totales que se ha obtenido (sin descuento)
# 2.-Saber que artículos no son de temporada, y cuanto se obtuvo (€) por ellos
# 3.-Calcular porcentaje de prendas por estación ¿en qué estación se vende más?
# 4.-Artículos con la palabra cozy en el nombre
# 5.-Calcular nueva columna con el descuento aplicado, y valorar la diferencia de ganancias respecto al artículo sin descuento
# 6.-Mayor país proveedor, e ingresos por país, tenemos que disolver el contrato con el país con menores ingresos.
# 7.-¿Cuál es el color favorito de la temporada? ¿y del invierno?
# 8.-Marca con mayores ventas --> Para enviar detalle por parte de la empresa
# 9.-Hacer una relación de ventas e inventario, para eliminar 14 prendas de nuestro stock.
# 10.- Elaborar un informe final.

In [36]:
print(f"El número de filas que tenemos es {df_excel.shape[0]}, y el número de columnas es {df_excel.shape[1]}")

El número de filas que tenemos es 115, y el número de columnas es 11


In [37]:
# Conociendo los objetivos del estudio, lo primero que habrá que hacer es una limpieza de nuestros datos:

df_excel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 115 entries, nan to nan
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             114 non-null    object
 1   producto       115 non-null    object
 2   precio (€)     110 non-null    object
 3   descuento (%)  110 non-null    object
 4   color          103 non-null    object
 5   pais_origen    107 non-null    object
 6   temporada2025  104 non-null    object
 7   ventas_unidad  109 non-null    object
 8   inventario     99 non-null     object
 9   marca          102 non-null    object
 10  estacion       95 non-null     object
dtypes: object(11)
memory usage: 10.8+ KB


In [38]:
# Obtenemos la lista de columnas categóricas que tienen nulos
nulos_esta_cat = df_excel[df_excel.columns[df_excel.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['id', 'precio (€)', 'descuento (%)', 'color', 'pais_origen',
       'temporada2025', 'ventas_unidad', 'inventario', 'marca', 'estacion'],
      dtype='object')


In [39]:
df_excel.describe(include = "object").T

,count,unique,top,freq
id,114,113,2,2
producto,115,100,Polo Urban Fit,2
precio (€),110,79,20,4
descuento (%),110,12,5,23
color,103,10,azul,19
pais_origen,107,8,ES,30
temporada2025,104,7,No,36
ventas_unidad,109,29,21,7
inventario,99,48,45,7
marca,102,13,D,30


In [ ]:
# como hemos visto lo anterior nos devuelve una serie de booleanos lo cual es poco entendible, 
# por lo que añadiremos el método '.sum()' para que nos cuente cuantas filas tenemos duplicadas
# en este caso el código nos esta diciendo que no tenemos filas duplicadas

df_excel.duplicated().sum() # No tenemos valores duplicados(PERO OJO EL id 2 está repetido.)

np.int64(0)

In [ ]:
# id: Serie de números enteros sin alteración. Tipo objeto --> convertir a int (aunque no necesario, es un identificador) 

In [41]:
# producto: Todo bien, un str.

df_excel["producto"].nunique() # cuenta el número de valores únicos en la columna "producto"

# Hay prendas con el mismo nombre, de momento no sabemos si datos repetidos, o quizás tienen diferentes características

100

In [ ]:
# precio (€):